In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-08-06 16:14:20--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  29.6MB/s    in 0.8s    

2025-08-06 16:14:21 (29.6 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code
# Filter users with >= 200 ratings
user_counts = df_ratings['user'].value_counts()
df_ratings_filtered = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

# Filter books with >= 100 ratings
book_counts = df_ratings_filtered['isbn'].value_counts()
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(book_counts[book_counts >= 100].index)]


In [5]:
# Merge to get book titles
df_merged = pd.merge(df_ratings_filtered, df_books, on='isbn')

# Create pivot table: rows=books, columns=users, values=ratings
book_user_matrix = df_merged.pivot_table(index='title', columns='user', values='rating').fillna(0)

# Convert to sparse matrix (optional but recommended)
book_user_sparse_matrix = csr_matrix(book_user_matrix.values)


In [6]:
# Fit the model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse_matrix)

# Store book titles in list for reference
book_titles = book_user_matrix.index.tolist()


In [7]:
def get_recommends(book = ""):
    # Get index of the book
    if book not in book_titles:
        return [book, []]

    idx = book_titles.index(book)

    # Get distances and indices of 6 neighbors (including the book itself)
    distances, indices = model.kneighbors(book_user_matrix.iloc[idx, :].values.reshape(1, -1), n_neighbors=6)

    # Skip the first one because it's the input book
    recommended_books = []
    for i in range(1, len(distances[0])):
        recommended_books.append([book_titles[indices[0][i]], distances[0][i]])

    return [book, recommended_books]


In [8]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', np.float32(0.7234864)], ["The Pilot's Wife : A Novel", np.float32(0.81926787)], ['The Joy Luck Club', np.float32(0.81986046)], ['The Notebook', np.float32(0.82366824)], ['Bel Canto: A Novel', np.float32(0.8247875)]]]
You haven't passed yet. Keep trying!
